# Setup

In [1]:
import numpy as np
import os
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPool1D, Dropout, Dense, GlobalMaxPool1D, Embedding, Activation, Flatten, BatchNormalization
import  as tfds
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn import preprocessing
from tqdm.auto import tqdm
tqdm.pandas()
import nltk
nltk.download('stopwords')
nltk.download('punkt')

tfds.disable_progress_bar()
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices('GPU'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/device:GPU:0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import torch

def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
!wget -O parler_annotated_data.csv https://github.com/NasLabBgu/parler-hate-speech/raw/main/parler_annotated_data.csv

# Data Preperation

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_CLASSES = 4

In [ ]:
df = pd.read_csv('parler_annotated_data.csv')
print(df.describe())
df

In [7]:
def regression_to_labels(x):
  if x >= 3.0:
    return 1
  elif x <= 2.0:
    return 0
  return -1

text_df = df.copy()
text_df['label'] = text_df['label_mean'].map(regression_to_labels)
print(text_df['label'].value_counts())
text_df = text_df[text_df['label'] != -1][['text', 'label']]
text_df

 0    4452
 1    4068
-1    1601
Name: label, dtype: int64


,text,label
0,Black men are taught that a word justifies all...,1
1,Biden and his supporters are claiming that Bid...,1
2,Pelosi; âOne way or another Joe Biden will b...,0
3,Northern California Declares Independence From...,1
4,Whistleblower claims top DHS officials sought ...,0
...,...,...
10116,Minneapolis City Council Votes To Eliminate Po...,0
10117,And just like that facebook is gone,0
10118,Obama Says He Will Campaign For Biden If He Ca...,0
10119,Shoup Voting Machine,0


In [8]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(text_df, test_size=0.2, stratify=text_df['label'])

# BART Paraphrasing

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

bart = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

def bart_paraphrase(sent):
  input_ids = bart_tokenizer(sent, return_tensors="pt")['input_ids']
  outputs = model.generate(input_ids,
                           num_beams=2,
                           min_length=int(0.8 * len(sent)),
                           max_length=int(1.2 * len(sent)))
  decoded_outputs = bart_tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  return [dx for dx in set(decoded_outputs) if dx.lower() != sent.lower()]

# T5 Paraphrasing

In [ ]:
from transformers import T5ForConditionalGeneration,T5Tokenizer

t5 = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser').to(device)
t5_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')

def t5_paraphrase(sent):
  encoding = t5_tokenizer.encode_plus(f"paraphrase: {sent} </s>", return_tensors="pt")
  outputs = t5.generate(
    input_ids=encoding["input_ids"].to(device),
    attention_mask=encoding["attention_mask"].to(device),
    do_sample=True,
    max_length=256,
    top_k=50,
    top_p=0.99,
    early_stopping=True,
    num_return_sequences=10
  )

  decoded_outputs = t5_tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  return [dx for dx in set(decoded_outputs) if dx.lower() != sent.lower()]

# Filter By Sentiment

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

hate_tokenizer = AutoTokenizer.from_pretrained("tomh/toxigen_hatebert")
hate_model = AutoModelForSequenceClassification.from_pretrained("tomh/toxigen_hatebert")

def classify_hate(texts):
  hate_model(**hate_tokenizer(texts))
  most_extreme_idx = classify_hate(paraphrases).argmax(axis=row['label'])
  return paraphrases[most_extreme_idx]
  return hate_model(**hate_tokenizer(sents))

In [50]:


train_df['paraphrase'] = train_df.progress_apply(augment_paraphrase, axis=1)

  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


2331    [How has the government compromised ties to Tr...
9260    [Why do the Maoists burn down New Delhi?, Who ...
3259    [Is BLM a disgusting, violent, Marxist Organiz...
6980    [After Trump re-election, we should give Kamal...
3199    [What is the gun that shoots round corners â  ...
Name: text, dtype: object

In [ ]:
most_extreme_idx = classify_hate(paraphrases).argmax(axis=row['label'])
  return paraphrases[most_extreme_idx]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

hate_tokenizer = AutoTokenizer.from_pretrained("tomh/toxigen_hatebert")

hate_model = AutoModelForSequenceClassification.from_pretrained("tomh/toxigen_hatebert")

def classify_hate(sents):
  return hate_model(**hate_tokenizer(sents))

In [ ]:
org_train_df = train_df[['text', 'label']]
org_train_df['is_t5_paraphrased'] = False
aug_train_df = train_df[['paraphrase', 'label']].rename(columns={'paraphrase': 'text'})
aug_train_df['is_t5_paraphrased'] = True
full_train_df = pd.concat(org_train_df, aug_train_df).reset_index().shuffle(1)

# Bert Fine-Tune

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

clf_tokenizer = AutoTokenizer.from_pretrained("tomh/toxigen_hatebert")

clf = AutoModelForSequenceClassification.from_pretrained("tomh/toxigen_hatebert")



# Attack Bert